04/05/2023      
notebook     
objectif : faire le bilan sous forme de diagrammes en bâton cumulés et diagramme de Wenn pour le corpus anglais .    
contexte : suite de "testELSpacy_venn_baton2.ipynb"

# Evaluation comparative  du système d'entity linking de SpaCy

## I-

In [ ]:
import json
import spacy  # version 3.5
import glob
import pandas as pd

def lire_fichier(chemin):
    with open (chemin, "r", encoding="utf-8") as fichier:
        texte=fichier.read()
        return texte
def stocker(chemin, contenu):
    w =open(chemin, "w")
    w.write(json.dumps(contenu , indent = 2))
    w.close()

nlp = spacy.load("en_core_web_md")
nlp.add_pipe("entityLinker", last=True)

path_corpora = "C:/Users/dorle/Documents/EXPE00-CAROLINE-STAGE/ELTeC-eng_nonref_nonocr/*/*/*.txt"

for path in glob.glob(path_corpora):
    print(path)
    txt=lire_fichier(path)
    #doc = nlp(txt[:5000])  
    doc = nlp(txt) 
    i=0
    dico_sent_tok ={}
    
    
    for ent in doc._.linkedEntities: 
            ide = "ID"+str(i)
            url_00 = ent.get_url()
            label_00=ent.get_label()
            description=ent.get_description()
            span_00=ent.get_span()
            span_oo=str(span_00)
            doc2=nlp(span_oo)
            for en in doc2.ents:#Pour le typage avec les labels de l'entity linking
                
                # print(en.label_)
                dico_sent_tok[ide]={}
                dico_sent_tok[ide]["url"]=url_00
                dico_sent_tok[ide]["span"]=span_oo
                dico_sent_tok[ide]["label"]=label_00
                dico_sent_tok[ide]["description"]=description
                dico_sent_tok[ide]["type"]=en.label_#Pour le typage avec les labels de l'entity linking
                i+=1
            stocker("%s_entity-linker-en_core_web_md.json"%path,dico_sent_tok)

In [ ]:
import os
import json
from matplotlib import pyplot as plt
import numpy as np
from matplotlib_venn import venn2, venn2_circles
import glob

def Venn (chemin):
    # spécifiez le chemin vers votre dossier contenant les fichiers
    path_corpora = "../DATA/DATA-Fra_spaCy3.3.1_CONCAT/ADAM/*/*/*.json"

    # créez une liste pour stocker les ensembles d'URL communs
    liste = []
    liste_fichier = []

    # bouclez à travers tous les fichiers dans le dossier
    for nom_fichier in glob.glob(chemin):
        if "entity-linker" in str(nom_fichier):
            liste_fichier= ['Kraken', 'le texte de référence', 'Tesseract binarisé', 'le texte de référence', 'Tesseract PNG','le texte de référence', 'TesseractFra PNG', 'le texte de référence']
            # ouvrez le fichier et chargez les dictionnaires JSON
            with open(nom_fichier) as f:
                dictionnaires = json.load(f)

            # créez un ensemble pour les URL du fichier courant
            set_url = set()

            # bouclez à travers tous les dictionnaires dans le fichier
            for cle, dico in dictionnaires.items():
                # obtenez l'URL du dictionnaire
                url = dico.get("url")
                # ajoutez l'url à l'ensemble des URL
                set_url.add(url)

            # ajoutez l'ensemble d'URL du fichier courant à la liste
            liste.append(set_url)

    arr = None
    # bouclez à travers les paires d'ensembles d'URL dans la liste
    for i in range(len(liste) - 1):
        ocr = liste[i]
        nb_ocr = len(ocr)
        reference = liste[i+1]
        nb_reference = len (reference)
        intersection = ocr.intersection(reference)
        nb_intersection = len(intersection)
        element_propre_ocr = len([element for element in ocr if element not in reference]) 
        element_propre_reference = len([element for element in reference if element not in ocr])
        nom = liste_fichier[i]
        if arr is None:
            arr = np.array([nom, nb_ocr, nb_reference, nb_intersection, element_propre_ocr, element_propre_reference])
        else:
            arr = np.vstack((arr, [nom, nb_ocr, nb_reference, nb_intersection, element_propre_ocr, element_propre_reference]))

        # affiche les noms des fichiers
        print("Valeurs communes entre", liste_fichier[i], "et", liste_fichier[i+1] )
 

        # vérifie si les ensembles d'URL contiennent des éléments avant de générer le Venn diagram
        if ocr and reference:
            venn2([ocr, reference], set_labels=(str(liste_fichier[i]), str(liste_fichier[i+1])))
            plt.show()
        else:
            print("Ensemble d'URL vide pour au moins un fichier")

    print(arr)


In [13]:
import json
import spacy  # version 3.5
import glob
import pandas as pd
import matplotlib.pyplot as plt

def lire_fichier(chemin):
    with open (chemin, "r", encoding="utf-8") as fichier:
        texte=fichier.read()
        return texte
def stocker(chemin, contenu):
    w =open(chemin, "w")
    w.write(json.dumps(contenu , indent = 2))
    w.close()
def batons (chemin, modele) :
    url_communs = []
    liste_loc = []
    liste_autre = []
    liste_a = []
    liste_b = []

    nlp = spacy.load(modele)
    nlp.add_pipe("entityLinker", last=True)

    

    for path in glob.glob(chemin):
        #print(path)
        txt=lire_fichier(path)
        #doc = nlp(txt[:5000]) 
        doc = nlp(txt)
        i=0
        dico_sent_tok ={}
        liste_loc = []
        liste_autre = []
    
    
    
        for ent in doc._.linkedEntities: 
            ide = "ID"+str(i)
            url_00 = ent.get_url()
            label_00=ent.get_label()
            description=ent.get_description()
            span_00=ent.get_span()
            span_oo=str(span_00)
            doc2=nlp(span_oo)
            for en in doc2.ents:#Pour le typage avec les labels de l'entity linking
                
                # print(en.label_)
                dico_sent_tok[ide]={}
                dico_sent_tok[ide]["url"]=url_00
                dico_sent_tok[ide]["span"]=span_oo
                dico_sent_tok[ide]["label"]=label_00
                dico_sent_tok[ide]["description"]=description
                dico_sent_tok[ide]["type"]=en.label_#Pour le typage avec les labels de l'entity linking
                i+=1
                
                if dico_sent_tok[ide]["type"] == "LOC":
                    #dico_sent_tok[ide]["type"] = True
                    liste_loc.append(dico_sent_tok[ide])
                else:
                    #dico_sent_tok[ide]["type"] = False
                    liste_autre.append(dico_sent_tok)
        a=len(liste_loc)
        # nb de dictionnaire qui contiennent des loc
        b=len(liste_autre)
        #nb de dictionnaire qui ne contiennent pas des loc
        liste_a.append(a)
        liste_b.append(b)
        stocker("%s_entity-linker.json"%path,dico_sent_tok)
                    

    largeur_barre = 0.8


    x = range(len(liste_a)) # position en abscisse des barres

    # # Tracé

    plt.bar(x, liste_a, width = largeur_barre, color = "#3ED8C9")

    plt.bar(x, liste_b, width = largeur_barre, bottom = liste_a, color = "#EDFF91")

    plt.xticks(range(len(liste_a)), ['Kraken', 'référence', 'Tesseract binarisé', 'référence', 'Tesseract PNG','référence'], rotation=90)
    
    plt.show()

In [14]:
path_corpora = 'C:/Users/dorle/Documents/EXPE00-CAROLINE-STAGE/ELTeC-eng_nonref_nonocr/AGUILAR_home-influence/*/*.txt'
batons (path_corpora,"en_core_web_md")

ValueError: [E088] Text of length 1029995 exceeds maximum of 1000000. The parser and NER models require roughly 1GB of temporary memory per 100,000 characters in the input. This means long texts may cause memory allocation errors. If you're not using the parser or NER, it's probably safe to increase the `nlp.max_length` limit. The limit is in number of characters, so you can check whether your inputs are too long by checking `len(text)`.